In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time

from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation
from pgmpy.models import BayesianNetwork

import modelling_o2.o2satffa as o2satffa
import models.helpers as mh
import models.builders as mb
import inference.helpers as ih


plotsdir = "../../../../PlotsBreathe/O2_modelling/"

In [2]:
df = pd.read_excel(plotsdir + "/Breathe_O2_FEV1.xlsx")
# ID column as type string
df["ID"] = df["ID"].astype(str)
# Date Redocrded as datetime
df["Date Recorded"] = df["Date Recorded"].dt.date

In [3]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125


## Infer O2SatFFA after observing FEV1

In [4]:
def model_up_to_O2SatFFA(hfev1_prior, ho2sat_prior):
    """
    This is a point in time model with
    FEV1 = HFEV1 * (1-AR)
    O2SatFFA = HO2Sat * drop_func(AR)

    The model is the same as build_HFEV1_AB_FEV1(), with Airway Blockage renamed to Airway Resistance.
    """
    print("*** Building FEV1 and O2 point in time model ***")

    # Setting resolution of 0.05 to avoud rounding errors for AR
    HFEV1 = mh.VariableNode("Healthy FEV1 (L)", 1, 6, 0.05, prior=hfev1_prior)
    AR = mh.VariableNode("Airway resistance (%)", 0, 90, 1, prior={"type": "uniform"})
    ecFEV1 = mh.VariableNode("FEV1 (L)", 0, 6, 0.05, prior=None)
    # Lowest predicted FEV1 is 15% (AR = 1-predictedFEV1)
    HO2Sat = mh.VariableNode(
        "Healthy O2 saturation (%)", 90, 100, 0.5, prior=ho2sat_prior
    )
    # Highest drop is 93% (for AR = 90%), hence the lowest O2SatFFA is 90 * 0.93 = 83.7%
    O2SatFFA = mh.VariableNode(
        "O2 sat if fully functional alveoli (%)", 80, 100, 0.5, prior=None
    )

    prior_hfev1 = TabularCPD(
        variable=HFEV1.name,
        variable_card=HFEV1.card,
        values=HFEV1.cpt,
        evidence=[],
        evidence_card=[],
    )
    prior_ho2sat = TabularCPD(
        variable=HO2Sat.name,
        variable_card=HO2Sat.card,
        values=HO2Sat.cpt,
        evidence=[],
        evidence_card=[],
    )
    prior_ar = TabularCPD(
        variable=AR.name,
        variable_card=AR.card,
        values=AR.cpt,
        evidence=[],
        evidence_card=[],
    )
    cpt_fev1 = TabularCPD(
        variable=ecFEV1.name,
        variable_card=ecFEV1.card,
        values=mh.calc_pgmpy_cpt_X_x_1_minus_Y(HFEV1, AR, ecFEV1),
        evidence=[HFEV1.name, AR.name],
        evidence_card=[HFEV1.card, AR.card],
    )
    cpt_o2_sat_ffa = TabularCPD(
        variable=O2SatFFA.name,
        variable_card=O2SatFFA.card,
        values=o2satffa.calc_cpt(O2SatFFA, HO2Sat, AR, debug=False),
        evidence=[HO2Sat.name, AR.name],
        evidence_card=[HO2Sat.card, AR.card],
    )

    model = BayesianNetwork(
        [
            (HFEV1.name, ecFEV1.name),
            (AR.name, ecFEV1.name),
            (HO2Sat.name, O2SatFFA.name),
            (AR.name, O2SatFFA.name),
        ]
    )

    model.add_cpds(cpt_fev1, prior_ar, prior_hfev1, prior_ho2sat, cpt_o2_sat_ffa)

    model.check_model()
    inf_alg = BeliefPropagation(model)
    return (model, inf_alg, HFEV1, ecFEV1, AR, HO2Sat, O2SatFFA)

In [5]:
# Infer airway resistance using the model
for id in df.ID.unique():
    # for id in ["101"]:
    df_for_ID = df[df.ID == id].copy().reset_index()
    print(f"\nRunning for ID {id}, with {len(df_for_ID)} observations")
    # Take one element of the df for ID at column height
    height = df_for_ID.Height[0]
    sex = df_for_ID.Sex[0]
    hfev1_prior = {
        "type": "default",
        "height": height,
        "age": df_for_ID.Age[0],
        "sex": sex,
    }
    ho2sat_prior = {
        "type": "default",
        "height": height,
        "sex": sex,
    }
    tic = time.time()
    (
        model,
        inf_alg,
        HFEV1,
        ecFEV1,
        AR,
        HO2Sat,
        O2SatFFA,
    ) = model_up_to_O2SatFFA(hfev1_prior, ho2sat_prior)
    print(f"model took {time.time() - tic} seconds to build")

    tic = time.time()
    df_for_ID["AR from FEV1"] = np.nan
    df_for_ID["AR from ecFEV1"] = np.nan
    df_for_ID["O2SatFFA from FEV1"] = np.nan
    df_for_ID["O2SatFFA from ecFEV1"] = np.nan
    fev1s_tmp = []
    ecfev1s_tmp = []
    for i in range(len(df_for_ID)):
        fev1_obs = df_for_ID.loc[i, "FEV1"]
        ecfev1_obs = df_for_ID.loc[i, "ecFEV1"]

        # FEV1
        if fev1_obs in fev1s_tmp:
            df_for_ID.loc[i, "AR from FEV1"] = df_for_ID.loc[
                df_for_ID["FEV1"] == fev1_obs, "AR from FEV1"
            ].values[0]
            df_for_ID.loc[i, "O2SatFFA from FEV1"] = df_for_ID.loc[
                df_for_ID["FEV1"] == fev1_obs, "O2SatFFA from FEV1"
            ].values[0]
        else:
            res_ar_for_fev1 = ih.infer(
                inf_alg, [AR], [[ecFEV1, fev1_obs]], show_progress=False
            )
            df_for_ID.loc[i, "AR from FEV1"] = AR.get_mean(res_ar_for_fev1.values)

            res_o2satffa_for_fev1 = ih.infer(
                inf_alg, [O2SatFFA], [[ecFEV1, fev1_obs]], show_progress=False
            )
            df_for_ID.loc[i, "O2SatFFA from FEV1"] = O2SatFFA.get_mean(
                res_o2satffa_for_fev1.values
            )

        # ecFEV1
        if ecfev1_obs in ecfev1s_tmp:
            df_for_ID.loc[i, "AR from ecFEV1"] = df_for_ID.loc[
                df_for_ID["ecFEV1"] == ecfev1_obs, "AR from ecFEV1"
            ].values[0]

            df_for_ID.loc[i, "O2SatFFA from ecFEV1"] = df_for_ID.loc[
                df_for_ID["ecFEV1"] == ecfev1_obs, "O2SatFFA from ecFEV1"
            ].values[0]
        else:
            res_ar_for_ecfev1 = ih.infer(
                inf_alg, [AR], [[ecFEV1, ecfev1_obs]], show_progress=False
            )
            df_for_ID.loc[i, "AR from ecFEV1"] = AR.get_mean(res_ar_for_ecfev1.values)

            res_o2satffa_for_ecfev1 = ih.infer(
                inf_alg, [O2SatFFA], [[ecFEV1, ecfev1_obs]], show_progress=False
            )
            df_for_ID.loc[i, "O2SatFFA from ecFEV1"] = O2SatFFA.get_mean(
                res_o2satffa_for_ecfev1.values
            )

        fev1s_tmp = np.append(fev1s_tmp, fev1_obs)
        ecfev1s_tmp = np.append(ecfev1s_tmp, ecfev1_obs)
    print(f"inference took {time.time() - tic} seconds to run")

    # Add to df
    df.loc[df.ID == id, "AR from ecFEV1 (%)"] = df_for_ID["AR from ecFEV1"].values
    df.loc[df.ID == id, "AR from FEV1 (%)"] = df_for_ID["AR from FEV1"].values

    df.loc[df.ID == id, "O2SatFFA from ecFEV1 (%)"] = df_for_ID[
        "O2SatFFA from ecFEV1"
    ].values
    df.loc[df.ID == id, "O2SatFFA from FEV1 (%)"] = df_for_ID[
        "O2SatFFA from FEV1"
    ].values

# IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
# Happens when the results becomes too close to 0


Running for ID 101, with 816 observations
*** Building FEV1 and O2 point in time model ***


/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:24: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_hfev1 = TabularCPD(
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:31: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_ho2sat = TabularCPD(
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:38: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_ar = TabularCPD(
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:327: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position 

model took 46.833966970443726 seconds to build
inference took 23.40282392501831 seconds to run

Running for ID 102, with 151 observations
*** Building FEV1 and O2 point in time model ***


/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:24: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_hfev1 = TabularCPD(
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:31: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_ho2sat = TabularCPD(
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:38: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_ar = TabularCPD(
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:327: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position 

model took 46.56610608100891 seconds to build
inference took 11.48368501663208 seconds to run

Running for ID 103, with 511 observations
*** Building FEV1 and O2 point in time model ***


...

model took 38.31896781921387 seconds to build
inference took 0.33847784996032715 seconds to run

Running for ID 357, with 3 observations
*** Building FEV1 and O2 point in time model ***


/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:24: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_hfev1 = TabularCPD(
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:31: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_ho2sat = TabularCPD(
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:38: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_ar = TabularCPD(
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:327: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position 

model took 38.119850158691406 seconds to build
inference took 0.8445069789886475 seconds to run

Running for ID 358, with 11 observations
*** Building FEV1 and O2 point in time model ***


/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:24: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_hfev1 = TabularCPD(
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:31: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_ho2sat = TabularCPD(
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_48157/847093712.py:38: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_ar = TabularCPD(
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:327: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position 

model took 38.24770927429199 seconds to build
inference took 2.5634076595306396 seconds to run


In [6]:
df.to_excel(f"{plotsdir}airwayresistance_o2satffa_df.xlsx")